# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846885642593                   -0.70    4.8         
  2   -7.852318268849       -2.26       -1.53    1.0   24.9ms
  3   -7.852613446162       -3.53       -2.56    1.5   27.4ms
  4   -7.852645992697       -4.49       -2.89    2.5   34.6ms
  5   -7.852646512368       -6.28       -3.19    1.0   25.4ms
  6   -7.852646678965       -6.78       -4.03    1.0   25.6ms
  7   -7.852646686579       -8.12       -5.28    2.0   32.0ms
  8   -7.852646686725       -9.83       -5.49    2.0   31.0ms
  9   -7.852646686729      -11.49       -5.78    1.0   26.0ms
 10   -7.852646686730      -11.97       -6.70    1.0   26.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846817587579                   -0.70           4.5         
  2   -7.852522503696       -2.24       -1.64   0.80    2.0    268ms
  3   -7.852634551026       -3.95       -2.74   0.80    1.0   23.5ms
  4   -7.852646499566       -4.92       -3.23   0.80    2.2   31.7ms
  5   -7.852646672978       -6.76       -4.05   0.80    1.0   24.0ms
  6   -7.852646686348       -7.87       -4.79   0.80    1.5   26.9ms
  7   -7.852646686724       -9.42       -5.58   0.80    2.0   30.1ms
  8   -7.852646686730      -11.26       -7.01   0.80    1.8   27.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.400071e+01     3.795105e+00
 * time: 0.07839298248291016
     1     1.560965e+00     2.020678e+00
 * time: 0.326678991317749
     2    -1.830851e+00     2.305360e+00
 * time: 0.3531157970428467
     3    -3.768167e+00     2.137606e+00
 * time: 0.390211820602417
     4    -5.381710e+00     1.741120e+00
 * time: 0.42684292793273926
     5    -6.909084e+00     8.471153e-01
 * time: 0.46335792541503906
     6    -7.139883e+00     1.760889e+00
 * time: 0.4888319969177246
     7    -7.577181e+00     1.433182e+00
 * time: 0.5139379501342773
     8    -7.634387e+00     2.408454e+00
 * time: 0.538985013961792
     9    -7.684652e+00     9.908471e-01
 * time: 0.6826379299163818
    10    -7.707944e+00     5.566868e-01
 * time: 0.7287688255310059
    11    -7.770487e+00     1.367820e+00
 * time: 0.7533748149871826
    12    -7.784955e+00     8.293000e-01
 * time: 0.7778849601745605
    13    -7.816646e+00     9.276254e-02
 * time: 0.8237588405

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846801037395                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645892059                   -1.64         
  2   -7.852646686730       -6.10       -3.70    2.12s
  3   -7.852646686730      -13.27       -7.24    179ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.0193618008631192e-7
|ρ_newton - ρ_scfv| = 2.6049040122515025e-7
|ρ_newton - ρ_dm|   = 7.234332400427705e-10
